# Web Scrapping laboratório aplicada

In [1]:
import requests
from bs4 import BeautifulSoup

### Para nosso projeto iremos retirar os dados do site Project Gutenberg, este é um repositório de livros.

In [9]:
#url do site que vamos fazer o web scrapping iremos variar os livros de acordo com ordem alfabética
url = "https://www.gutenberg.org/browse/titles/a"

# Cabeçalho para simular um navegador
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [10]:
response = requests.get(url, headers=headers)
response # https://www.w3schools.com/tags/ref_httpmessages.asp -- 200 é bom

<Response [200]>

In [ ]:
soup = BeautifulSoup(response.text, "html.parser")
soup # eu não estou colocando as saidas pq elas tao mto grandes

#### Nesse ponto normalmente temos que encontrar padrões, nesse site, os livros dados da seguinte forma.

     <div class="pgdbbytitle">

     <h2><a href="/ebooks/66447">The Aab</a> (English)</h2>
     <p>by <a href="/browse/authors/l#a35976">Ludwig, Edward W.</a></p>

In [ ]:
bloco_livros = soup.find("div", class_ = "pgdbbytitle")

bloco_livros

In [ ]:
links = [a["href"] for a in bloco_livros.find_all("a", href = True)]

links

In [ ]:
# Na saída acima também vieram os links dos autores, que no caso não é algo que queremos então temos que tirar os que temos eles.

links_ebooks = [books for books in links if "ebooks" in books]
print(f"Foram obtidos {len(links_ebooks)} links de livros começando com letra A ")
links_ebooks

In [ ]:
# Agora já temos os links para abrir as informações de cada um dos livros que é da forma 
# https://www.gutenberg.org/ + {link dos ebooks}

In [27]:
base_url = "https://www.gutenberg.org"

full_url = base_url + links_ebooks[0]
response = requests.get(full_url, headers= headers)
response

<Response [200]>

In [28]:
full_url

'https://www.gutenberg.org/ebooks/66447'

In [ ]:
soup = BeautifulSoup(response.text, "html.parser")
soup

In [ ]:
print(soup.get_text(separator="\n").strip())

In [ ]:
titulo = soup.find('td', itemprop='headline').get_text(strip=True)
print(titulo)

autor = soup.find('a', itemprop = 'creator').get_text(strip = True)
print(autor)

# no caso da publicação original no html ela n tem um objeto tipo itemprop que tem nos outros então a gente acha o 
# Original publication que esta dentro de um th e dps pega o proximo objeto td dele
elemento_th = soup.find('th', string = lambda text: text and "Original Publication" in text)
elemento_td = elemento_th.find_next_sibling('td')
Publicacao = elemento_td.get_text(strip=True)
print(Publicacao)

#Mesma coisa do de cima
th_summary = soup.find('th', string = lambda text: text and "Summary" in text)
td_summary = th_summary.find_next_sibling('td')
resumo = td_summary.get_text(strip = True)
print(resumo)

th_idioma = soup.find('th', string = 'Language')
td_idioma = th_idioma.find_next_sibling('td')
idioma = td_idioma.get_text(strip = True)
print(idioma)

elementos = soup.find_all('td', property = 'dcterms:subject')
temas = [a.get_text(strip = True) for a in elementos]
print(temas)

categoria = soup.find('td', datatype = "dcterms:DCMIType").get_text(strip = True)
print(categoria)



In [63]:
# Agora vamos tentar pegar mta informação mas sem ferrar o site e tentar n tomar block

import time
import random
import string

In [64]:
letras = list(string.ascii_lowercase)

links_livros = []

for l in letras:
    url = f"https://www.gutenberg.org/browse/titles/{l}"
    response = requests.get(url, headers=headers)
    print(l, response)

    soup = BeautifulSoup(response.text, 'html.parser')
    bloco_livros = soup.find("div", class_ = "pgdbbytitle")

    links = [a['href'] for a in bloco_livros.find_all('a', href = True)]
    links_ebooks = [books for books in links if "ebooks" in books]

    links_livros += links_ebooks

    # Esperar alguns segundos antes de ir para a próxima página
    tempo_espera = random.uniform(0, 10)
    time.sleep(tempo_espera)

a <Response [200]>
b <Response [200]>
c <Response [200]>
d <Response [200]>
e <Response [200]>
f <Response [200]>
g <Response [200]>
h <Response [200]>
i <Response [200]>
j <Response [200]>
k <Response [200]>
l <Response [200]>
m <Response [200]>
n <Response [200]>
o <Response [200]>
p <Response [200]>
q <Response [200]>
r <Response [200]>
s <Response [200]>
t <Response [200]>
u <Response [200]>
v <Response [200]>
w <Response [200]>
x <Response [200]>
y <Response [200]>
z <Response [200]>


In [96]:
len(links_livros) # 102110 links é muito, portanto vou sortear 2500 aleatoriamente

random.seed(1122) 
links_sorteados = random.sample(links_livros, 2500)



In [104]:
#no fim decidi pegar mais links pra ficar com um dataset meio grande então vou pegar mais 2500
mais_links = [link for link in links_livros if link not in links_sorteados]

random.seed(1122) 
links_sorteados_2 = random.sample(mais_links, 2500) 
# so não esquecer de arrumar os filenames pra cima de 2501

In [ ]:
import json
import pathlib

data_dir = pathlib.Path("data")
data_dir.mkdir(exist_ok = True)

for i, link in enumerate(links_sorteados_2):

    base_url = "https://www.gutenberg.org"
    full_url = base_url + link
    response = requests.get(full_url, headers=headers)

    soup = BeautifulSoup(response.text, "html.parser")

    categoria = soup.find('td', datatype = "dcterms:DCMIType").get_text(strip = True)

    if categoria == "Text":

        titulo = soup.find('td', itemprop='headline')
        if titulo != None:
            titulo = titulo.get_text(strip=True)
        else:
            titulo = "Não foi possível obter"

        autor = soup.find('a', itemprop = 'creator')
        if autor != None:
            autor = autor.get_text(strip = True)
        else:
            autor = "Não foi possível obter"

        # no caso do sumario no html ele n tem um objeto tipo itemprop que tem nos outros então a gente acha o 
        # Original publication que esta dentro de um th e dps pega o proximo objeto td dele
        th_summary = soup.find('th', string = lambda text: text and "Summary" in text)
        if th_summary != None:
            td_summary = th_summary.find_next_sibling('td')
            resumo = td_summary.get_text(strip = True)
        else:
            resumo = "Não foi possível obter"

        th_idioma = soup.find('th', string = 'Language')
        if th_idioma != None:
            td_idioma = th_idioma.find_next_sibling('td')
            idioma = td_idioma.get_text(strip = True)
        else:
            idioma = "Não foi possível obter"

        elementos = soup.find_all('td', property = 'dcterms:subject')
        if elementos == None:
            temas = "Não foi possível obter"
        else:
            temas = [a.get_text(strip = True) for a in elementos]

        informacoes = {"url": full_url,
                    "autor": autor,
                    "titulo": titulo,
                    # "publicacao": Publicacao
                    "resumo": resumo,
                    "idioma": idioma,
                    "temas": temas
                    }
    
        filename = f"livro {i}.json"
        file_path = data_dir / filename
        with file_path.open('w', encoding= 'utf-8') as f:
            json.dump(informacoes, f, ensure_ascii = False, indent = 4)

        print(f'Arquivo salvo: {file_path}')

        tempo_espera = random.uniform(0, 1.5)
        time.sleep(tempo_espera)
    
    else:
        tempo_espera = random.uniform(0, 1.5)
        time.sleep(tempo_espera)



# Iniciando a análise
 -- Temos como variáveis autor, título, publicação, idioma, resumo e temas
 -- Eu fui analisar os dados e vi que eu esqueci de colocar a publicação no scrapping então ficou tudo com a mesma publicação daquela q eu peguei a primeira vez então vou tirar da análise ela.

In [1]:
import matplotlib as plt
import json
import pathlib
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob


In [ ]:
diretorio = pathlib.Path("data")

lista_dados = []

for livro in diretorio.rglob("*.json"):
    with livro.open('r', encoding='utf-8') as f:
        dados = json.load(f)
        lista_dados.append(dados)
            
lista_dados



In [3]:
dados = pd.DataFrame(lista_dados)
dados = dados.drop("publicacao", axis = 1)
dados

,url,autor,titulo,resumo,idioma,temas
0,https://www.gutenberg.org/ebooks/66692,"Kóbor, Tamás, 1867-1942",Munka : Elbeszélések,"""Munka: Elbeszélések"" by Tamás Kóbor is a coll...",Hungarian,"[Short stories, Hungarian, Hungarian fiction]"
1,https://www.gutenberg.org/ebooks/16837,"Stevenson, Robert Louis, 1850-1894",Ryöstölapsi: Kertomus David Balfourin seikkail...,"""Ryöstölapsi: Kertomus David Balfourin seikkai...",Finnish,"[Historical fiction, Scotland -- History -- 18..."
2,https://www.gutenberg.org/ebooks/70923,"Butler, Ellis Parker, 1869-1937",An experiment in gyro-hats,"""An Experiment in Gyro-Hats"" by Ellis Parker B...",English,"[Science fiction, Short stories, Humorous stor..."
3,https://www.gutenberg.org/ebooks/43473,"Aimard, Gustave, 1818-1883","The Trappers of Arkansas; or, The Loyal Heart","""The Trappers of Arkansas; or, The Loyal Heart...",English,"[Western stories, Frontier and pioneer life --..."
4,https://www.gutenberg.org/ebooks/58538,"Church, J. (John), 1780?-1825?",Is This Your Likeness? A Scriptural Definition...,"""Is This Your Likeness? A Scriptural Definitio...",English,[Christian life]
...,...,...,...,...,...,...
4931,https://www.gutenberg.org/ebooks/64576,"Seneca, Lucius Annaeus, 5? BCE-65","Minor Dialogues, Together With the Dialogue on...","""Minor Dialogues, Together With the Dialogue o...",English,"[Political ethics -- Early works to 1800, Educ..."
4932,https://www.gutenberg.org/ebooks/533,"Stratton-Porter, Gene, 1863-1924",The Song of the Cardinal,"""The Song of the Cardinal"" by Gene Stratton-Po...",English,"[Birds, Nature study, Cardinals (Birds)]"
4933,https://www.gutenberg.org/ebooks/1640,"MacDonald, George, 1824-1905",Lilith: A Romance,"""Lilith: A Romance"" by George MacDonald is a f...",English,[Fantasy fiction]
4934,https://www.gutenberg.org/ebooks/55602,"Bickerdyke, John, 1858-1933",The Curiosities of Ale & Beer: An Entertaining...,"""The Curiosities of Ale & Beer: An Entertainin...",English,"[Drinking customs, Drinking songs]"


In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4936 entries, 0 to 4935
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     4936 non-null   object
 1   autor   4936 non-null   object
 2   titulo  4936 non-null   object
 3   resumo  4936 non-null   object
 4   idioma  4936 non-null   object
 5   temas   4936 non-null   object
dtypes: object(6)
memory usage: 231.5+ KB


In [5]:
print(dados["idioma"].value_counts())# como a Base tem muitos livros em inglês iremos fazer uma flag do idioma para marcar esses casos
dados["Ingles"] = np.where(dados["idioma"] == "English",1,0)


idioma
English       4034
Finnish        233
French         232
German         134
Italian         68
Dutch           61
Spanish         46
Hungarian       40
Portuguese      38
Swedish         14
Esperanto        8
Latin            6
Norwegian        4
Tagalog          4
Polish           3
Frisian          2
Icelandic        1
Czech            1
Occitan          1
Cebuano          1
Danish           1
Galician         1
Breton           1
Welsh            1
Arabic           1
Name: count, dtype: int64


In [7]:
print(f"Foram encontrados {sum(dados["autor"] == "Não foi possível obter")} livros sem autores")
print(f"Foram encontrados {sum(dados["idioma"] == "Não foi possível obter")} livros sem idiomas")
print(f"Foram encontrados {sum(dados["resumo"] == "Não foi possível obter")} livros sem resumos")
print(f"Foram encontrados {sum(dados["temas"] == "Não foi possível obter")} livros sem tema")
print(f"Foram encontrados {sum(dados["titulo"] == "Não foi possível obter")} livros sem título")

#Como eu que coletei os dados e eu faço o que eu quero com eles eu vou deletar todo mundo q eu n achei pelo menos 
#uma informação, mas eu poderia criar flag pra esses casos e so filtras eles, e também as outras colunas tão preenchidas
# então n vai impactar nelas também, mas vou deletar aki.

indices = dados[(dados["autor"] == "Não foi possível obter") | (dados["resumo"] == "Não foi possível obter")].index

dados.drop(indices)
dados.reset_index(drop=True, inplace=True)
dados.head(5)

Foram encontrados 5 livros sem autores
Foram encontrados 0 livros sem idiomas
Foram encontrados 31 livros sem resumos
Foram encontrados 0 livros sem tema
Foram encontrados 0 livros sem título


,url,autor,titulo,resumo,idioma,temas,Ingles
0,https://www.gutenberg.org/ebooks/66692,"Kóbor, Tamás, 1867-1942",Munka : Elbeszélések,"""Munka: Elbeszélések"" by Tamás Kóbor is a coll...",Hungarian,"[Short stories, Hungarian, Hungarian fiction]",0
1,https://www.gutenberg.org/ebooks/16837,"Stevenson, Robert Louis, 1850-1894",Ryöstölapsi: Kertomus David Balfourin seikkail...,"""Ryöstölapsi: Kertomus David Balfourin seikkai...",Finnish,"[Historical fiction, Scotland -- History -- 18...",0
2,https://www.gutenberg.org/ebooks/70923,"Butler, Ellis Parker, 1869-1937",An experiment in gyro-hats,"""An Experiment in Gyro-Hats"" by Ellis Parker B...",English,"[Science fiction, Short stories, Humorous stor...",1
3,https://www.gutenberg.org/ebooks/43473,"Aimard, Gustave, 1818-1883","The Trappers of Arkansas; or, The Loyal Heart","""The Trappers of Arkansas; or, The Loyal Heart...",English,"[Western stories, Frontier and pioneer life --...",1
4,https://www.gutenberg.org/ebooks/58538,"Church, J. (John), 1780?-1825?",Is This Your Likeness? A Scriptural Definition...,"""Is This Your Likeness? A Scriptural Definitio...",English,[Christian life],1


In [8]:
# como os resumos são gerados por uma IA diretamente eles também estão todos em inglês então não vou precisar me preocupar
#com diferentes linguas durante a análise
dados[dados["Ingles"] == 0]["resumo"]

0       "Munka: Elbeszélések" by Tamás Kóbor is a coll...
1       "Ryöstölapsi: Kertomus David Balfourin seikkai...
11      "Páginas sevillanas" by Manuel Chaves Rey is a...
13      "Deberes de buena sociedad" by Camilo Fabra is...
18      "Puphejmo: Dramo en tri aktoj" by Henrik Ibsen...
                              ...                        
4907    "Rakkaita muistoja" by Joel Lehtonen is a coll...
4909    "Il romanzo della morte" by Bruno Sperani is a...
4919    "Naapurit: Alkuperäinen kertomus" by John Sten...
4921    "O Primo Bazilio: Episodio Domestico" by Eça d...
4930    "Herra Byronin ammatti" by Bernard Shaw is a n...
Name: resumo, Length: 902, dtype: object

# Início das análises
    -- Vamos realizar uma análise de sentimeto nos resumos dos textos (lembrando que eles foram gerados por uma IA, o que talvez facilite nossa analise? E depois ver se esse sentimento condiz com os temas que estão nos livros.)
    -- Vamos dar também um contexto da biblioteca que vamos usar. Vamos usar a biblioteca TextBlob. A análise de spentimento resulta em dois valores um sobre o sentimento (-1 a 1, sendo -1 negativo e 1 positivo). O segundo valore é sobre a subjetividade sendo 0 mais objetivo e 1 subjetivo

In [9]:
# Vamos fazer uma função que chama para realizar a análise de sentimento dos textos

def analise_sentimento(resumo):
    sentimento = TextBlob(resumo)
    return(sentimento.sentiment) # retorna uma lista com 2 valores. O 1 é sentimento e a entrada 2 é subjetividade

In [10]:
analise_resumos = dados["resumo"].apply(analise_sentimento) 

In [11]:
analise_resumos = dados['resumo'].apply(analise_sentimento).tolist()

for i in range(len(analise_resumos)):
    dados.loc[i, "sentimento"] = analise_resumos[i][0]
    dados.loc[i, "subjetividade"] = analise_resumos[i][1]

In [12]:
# Confesso que nessa parte aqui usei um pouco de GPT, pq eu queria tirar os sentimentos médio por tema e fazer umas
# análises nesse estilo e eu n tinha idéia de como fazer
temas = dados.explode('temas')
temas

# Então basicamente o que foi feito aqui foi duplicar as linhas expandindo todos os temas possiveis delas ai vamos ter
#uma linha com um tema por história.

,url,autor,titulo,resumo,idioma,temas,Ingles,sentimento,subjetividade
0,https://www.gutenberg.org/ebooks/66692,"Kóbor, Tamás, 1867-1942",Munka : Elbeszélések,"""Munka: Elbeszélések"" by Tamás Kóbor is a coll...",Hungarian,"Short stories, Hungarian",0,0.018056,0.488889
0,https://www.gutenberg.org/ebooks/66692,"Kóbor, Tamás, 1867-1942",Munka : Elbeszélések,"""Munka: Elbeszélések"" by Tamás Kóbor is a coll...",Hungarian,Hungarian fiction,0,0.018056,0.488889
1,https://www.gutenberg.org/ebooks/16837,"Stevenson, Robert Louis, 1850-1894",Ryöstölapsi: Kertomus David Balfourin seikkail...,"""Ryöstölapsi: Kertomus David Balfourin seikkai...",Finnish,Historical fiction,0,-0.085000,0.521111
1,https://www.gutenberg.org/ebooks/16837,"Stevenson, Robert Louis, 1850-1894",Ryöstölapsi: Kertomus David Balfourin seikkail...,"""Ryöstölapsi: Kertomus David Balfourin seikkai...",Finnish,Scotland -- History -- 18th century -- Fiction,0,-0.085000,0.521111
1,https://www.gutenberg.org/ebooks/16837,"Stevenson, Robert Louis, 1850-1894",Ryöstölapsi: Kertomus David Balfourin seikkail...,"""Ryöstölapsi: Kertomus David Balfourin seikkai...",Finnish,Kidnapping -- Fiction,0,-0.085000,0.521111
...,...,...,...,...,...,...,...,...,...
4935,https://www.gutenberg.org/ebooks/10624,"Blackwood, Algernon, 1869-1951",Three John Silence Stories,"""Three John Silence Stories"" by Algernon Black...",English,"Detective and mystery stories, English",1,0.177941,0.529412
4935,https://www.gutenberg.org/ebooks/10624,"Blackwood, Algernon, 1869-1951",Three John Silence Stories,"""Three John Silence Stories"" by Algernon Black...",English,"Ghost stories, English",1,0.177941,0.529412
4935,https://www.gutenberg.org/ebooks/10624,"Blackwood, Algernon, 1869-1951",Three John Silence Stories,"""Three John Silence Stories"" by Algernon Black...",English,"Silence, John (Fictitious character) -- Fiction",1,0.177941,0.529412
4935,https://www.gutenberg.org/ebooks/10624,"Blackwood, Algernon, 1869-1951",Three John Silence Stories,"""Three John Silence Stories"" by Algernon Black...",English,"Paranormal fiction, English",1,0.177941,0.529412


##### Se olharmos os temas dos textoAqui vemos um problema. Tem muitos temas diferentes então vamos mudar a abordagem e tentar diminuir essa quantidade de temas antes de fazer essa análise, tratando a coluna temas nos dados originais.

In [13]:
# vamos começar primeiramente criando uma lista com os generos de livros que consigo pensar e o GPT também 
# Pedi pro GPT criar os mais comuns e adicionei alguns q eu achei q apareciam em uns temas q eu vi de olho mesmo.
generos = [
    "comedy", "drama", "horror", "science fiction", "fantasy", "mystery", "romance", 
    "animation", "documentary", "biography", "sport", "war", "family", "crime", "historical", "supernatural", 
    "experimental", "cult", "religious", "noir", "short stories", "periodicals", "poetry", "fiction",
    "personal narratives", "witchcraft", "description", "western", "nature", "political", "tales",
    "humorous stories", "history", "war", "ghost", "literature", "adventure", "american", "socialism"]



In [14]:
# basicamente eu to recriando as listas de temas, mas com os generos q eu fiz
temas_tratados = []

for j in dados["temas"]: 
    generos_tema = [] 
    if isinstance(j, list): 
        for t in j: 
            for i in generos: 
                if i.lower() in t.lower():  
                    generos_tema.append(i.lower())
                    generos_tema = list(dict.fromkeys(generos_tema)) # tira os duplicados pq dicionario n pode ter duplicado
    if generos_tema == []:
        temas_tratados.append(["sem tema"])
    else:
        temas_tratados.append(generos_tema)  

print(temas_tratados, "\n", len(temas_tratados)) # vou deixar sem o output pra n ferrar o git tbm

[['short stories', 'fiction'], ['historical', 'fiction', 'history'], ['science fiction', 'fiction', 'short stories', 'humorous stories'], ['western', 'fiction'], ['sem tema'], ['history'], ['sem tema'], ['poetry', 'american'], ['war', 'history'], ['sem tema'], ['history'], ['history'], ['sem tema'], ['sem tema'], ['sem tema'], ['fiction'], ['poetry'], ['war', 'history'], ['drama'], ['sem tema'], ['sem tema'], ['fiction'], ['mystery', 'fiction'], ['description'], ['description'], ['sem tema'], ['history', 'poetry'], ['short stories'], ['witchcraft'], ['war', 'personal narratives'], ['periodicals'], ['description'], ['fiction'], ['war', 'history'], ['fiction'], ['sem tema'], ['history'], ['sem tema'], ['poetry'], ['sem tema'], ['drama'], ['tales', 'literature'], ['history', 'biography'], ['poetry'], ['fiction'], ['short stories', 'fiction'], ['fiction', 'adventure', 'history'], ['fiction'], ['drama'], ['fiction'], ['western', 'adventure', 'fiction'], ['sem tema'], ['literature', 'history

In [16]:
dados["temas_trat"] = temas_tratados

In [18]:
temas = dados.explode('temas_trat')

In [19]:
temas.groupby("temas_trat")["url"].count().sort_values(ascending=False)

temas_trat
fiction                1929
sem tema               1218
history                 658
short stories           334
periodicals             271
war                     270
poetry                  260
science fiction         259
literature              253
american                229
description             193
drama                   191
biography               156
adventure               135
mystery                 118
tales                    80
historical               70
fantasy                  59
western                  47
humorous stories         42
cult                     34
personal narratives      29
romance                  29
political                24
family                   22
ghost                    19
religious                16
sport                    13
horror                   13
comedy                   13
crime                    12
socialism                11
nature                   10
supernatural              6
witchcraft                6
Name: url

In [20]:
#vamos refazer aquela analise aumentando o data frame criando um linha pra cada tema por história
temas = dados.explode("temas_trat")

sentimento_por_tema = temas.groupby('temas_trat')['sentimento'].mean().reset_index()

print(sentimento_por_tema.sort_values(by='sentimento', ascending=False).head(5))
print(sentimento_por_tema.sort_values(by='sentimento', ascending=False).tail(5))


          temas_trat  sentimento
24           romance    0.143448
21            poetry    0.123121
30      supernatural    0.117915
16        literature    0.115557
15  humorous stories    0.114019
         temas_trat  sentimento
25  science fiction    0.041341
17          mystery    0.036588
4             crime    0.028844
34       witchcraft   -0.005373
14           horror   -0.033411


##### Conseguimos ver que apesar de a análise de sentimento colocar pouco peso nos sentimentos, isso também pode ser decorrente de os resumos serem gerados por uma IA e tentarem ser feitos da maneira mais imparcial possível, porém conseguimos perceber que os sentimentos mais negativos (que no casor receberam nota para serem neutros) tendem a concetrar temas mais negativos ou que não são para todos os publicos como horror, mistério, crimes. Quando olhamos os mais positivos conseguimos ver romance, poesia literatura e histórias de humor. Portanto mesmo com limitações parece que conseguimos ver uma separação entre os temas.

In [21]:
#vamos agora analisar a objetividade por tema e ver se isso pode ser um dos causadores de os sentimentos serem baixos.

subjetividade_por_tema = temas.groupby('temas_trat')['subjetividade'].mean().reset_index()

print(subjetividade_por_tema.sort_values(by="subjetividade", ascending=False).head(5))
print(subjetividade_por_tema.sort_values(by="subjetividade", ascending=False).tail(5))


          temas_trat  subjetividade
31             tales       0.547830
9            fantasy       0.543200
15  humorous stories       0.537224
25   science fiction       0.531131
14            horror       0.523099
             temas_trat  subjetividade
6           description       0.399682
13              history       0.383613
23            religious       0.383572
20  personal narratives       0.378630
28            socialism       0.374635


##### Analisando a subjetividade, conseguimos perceber que nenhum dos resumos é muito objetivo dado que o menor deles é 0.37 e o maior 0.54 que é praticamente neutro, portanto acredita-se que isso pode afetar um pouco a análise de sentimento realizada. No geral não foi possível obter tantas conclusões com a análise realizada, pois os valores da análise de sentimento do texto foram baixos. Porém também devemos levar me consideração que os resumos foram escritos por uma IA e tentam ser imparciais, de maneira que apenas digam o que se esperar da história, portanto a neutralidade obtida pode ser esperada.